In [2]:
!pip install pydub

In [34]:
!pip install ffmpeg

In [5]:
import boto3
from pydub import AudioSegment
import os
import io
import sys

In [17]:
def mono_to_stereo(file_path):
    
    left_channel = AudioSegment.from_wav(file_path)
    right_channel = AudioSegment.from_wav(file_path)

    stereo_sound = AudioSegment.from_mono_audiosegments(left_channel, right_channel)
    file_no_ext = os.path.splitext(file_path)[0]
    stereo_sound.export(file_no_ext+'_stereo.wav', format="wav")
    return file_no_ext+'_stereo.wav'

In [17]:
def process_audio(audio):
    
    left_channel = AudioSegment.from_wav(audio)
    right_channel = AudioSegment.from_wav(audio)

    stereo_sound = AudioSegment.from_mono_audiosegments(left_channel, right_channel)
    file_no_ext = os.path.splitext(file_path)[0]
    stereo_sound.export(file_no_ext+'_stereo.wav', format="wav")
    return file_no_ext+'_stereo.wav'

In [68]:
s3c = boto3.client('s3')
s3r = boto3.resource('s3')

bucket = s3r.Bucket('audio-classifier-data')

for obj in bucket.objects.filter(Prefix='processed/'):
    if obj.size:
        bucket.download_file(str(obj.key), str(obj.key))

FileNotFoundError: [Errno 2] No such file or directory: 'processed/eastern-whipbird/1d4bed34-4f47-4f8f-903f-ee1f88f28efa.wav.7C22a409'

In [29]:
s3r = boto3.resource('s3')

# Select Your S3 Bucket
bucket = s3r.Bucket('audio-classifier-data')

# Iterate All Objects in Your S3 Bucket Over the for Loop
for s3_object in bucket.objects.all():
    path, filename = os.path.split(s3_object.key)
    os.makedirs(path) 
    bucket.download_file(s3_object.key, path+"/"+filename)

FileExistsError: [Errno 17] File exists: 'clips'

In [85]:
#emma 
#this downloads the resources 
s3r = boto3.resource('s3')

# Select Your S3 Bucket
bucket = s3r.Bucket('audio-classifier-data')

# Iterate All Objects in Your S3 Bucket Over the for Loop
for s3_object in bucket.objects.all():
    path, filename = os.path.split(s3_object.key)
    print(path +"/"+filename)
    if not os.path.isdir(path): 
        os.mkdir(path)
    if filename:
        bucket.download_file(s3_object.key, path+"/"+filename)

        

clips/
mel-spectrogram/
processed/
processed/eastern-whipbird/
processed/eastern-whipbird/1d4bed34-4f47-4f8f-903f-ee1f88f28efa.wav
processed/eastern-whipbird/6316ecca-4b96-4477-8da7-e764429fa389.wav
processed/eastern-whipbird/aad78fc7-f674-4d44-af53-d3d25083420e.wav
processed/eastern-whipbird/b1a5da01-8a74-4abb-8cce-c68c31dc0a60.wav
processed/eastern-whipbird/b551a17d-28d5-4ee1-b5c1-19caa98ae514.wav
processed/eastern-whipbird/bf8d74b6-9763-4978-af5e-dda940d8f5f1.wav
processed/eastern-whipbird/dc6e3770-220e-43fe-9e84-dba5aa9b9770.wav
processed/eastern-whipbird/e409f4db-c60c-4d94-afbe-e7c3ba8db79c.wav
processed/kookaburra/
processed/kookaburra/08e85865-c479-4032-a63f-e5d36d4eb154.wav
processed/kookaburra/0f3df10f-be4a-49b0-b70a-54b174c7e015.wav
processed/kookaburra/249f605c-e971-4cf6-9b7e-aa6e95981863.wav
processed/kookaburra/67208ba1-56e8-43b2-af56-c7f36a015a95.wav
processed/kookaburra/718efcf9-5cd0-4aff-b7c4-5b6515b8f5be.wav
processed/kookaburra/7cbc1cd6-9114-4cc7-961b-7e0f336ad24d.wav

In [86]:
def mono_to_stereo(file_path):
    
    left_channel = AudioSegment.from_wav(file_path)
    right_channel = AudioSegment.from_wav(file_path)

    stereo_sound = AudioSegment.from_mono_audiosegments(left_channel, right_channel)
    file_no_ext = os.path.splitext(file_path)[0]
    stereo_sound.export(file_no_ext+'_stereo.wav', format="wav")
    return file_no_ext+'_stereo.wav' 

def slice_audio(filename, category):
    base = os.path.basename(filename)
    input_file = os.path.splitext(base)[0]
    extension = os.path.splitext(base)[1]
    audio = AudioSegment.from_file(filename)
    length = audio.duration_seconds
    if not os.path.isdir('clips/'+category): 
        os.mkdir('clips/'+category)
    for i in range(0, (int(length)-3)*1000, 500):
        start = i
        end = i+3000
        filename1 = input_file+'_'+str(start)+'_'+str(end)+extension
        #print(input_file)
        #print(filename1)
        newAudio = AudioSegment.from_wav(filename)
        newAudio = newAudio[start:end]
        newAudio.export('clips/'+category+'/'+filename1, format="wav")
        
def process_audio(file_path, category):
    sound = AudioSegment.from_file(file_path)
    channel_count = sound.channels
    if channel_count == 1: 
        file_path = mono_to_stereo(file_path)
    slice_audio(file_path, category)


In [87]:
import os
input_dir = 'processed/eastern-whipbird'

for subdir, dirs, files in os.walk(input_dir):
    for file in files:
        file_path = os.path.join(subdir, file)
        process_audio(file_path, 'curlew')
        

In [94]:
#send back to s3 

#from /clips send everything back up (currently just 'curlew' which is actually 'eastern-whipbird' but I forgot to change the naem)
path = 'clips'
for subdir, dirs, files in os.walk(path):
    for file in files:
        full_path = os.path.join(subdir, file)
        with open(full_path, 'rb') as data:
            bucket.put_object(Key=full_path, Body=data)